![ngo_project_image](ngo_project_image.jpg)

GoodThought NGO has been a catalyst for positive change, focusing its efforts on education, healthcare, and sustainable development to make a significant difference in communities worldwide. With this mission, GoodThought has orchestrated an array of assignments aimed at uplifting underprivileged populations and fostering long-term growth.

This project offers a hands-on opportunity to explore how data-driven insights can direct and enhance these humanitarian efforts. In this project, you'll engage with the GoodThought PostgreSQL database, which encapsulates detailed records of assignments, funding, impacts, and donor activities from 2010 to 2023. This comprehensive dataset includes:

- **`Assignments`:** Details about each project, including its name, duration (start and end dates), budget, geographical region, and the impact score.
- **`Donations`:** Records of financial contributions, linked to specific donors and assignments, highlighting how financial support is allocated and utilized.
- **`Donors`:** Information on individuals and organizations that fund GoodThought’s projects, including donor types.

Refer to the below ERD diagram for a visual representation of the relationships between these data tables:
<img src="erd.png" alt="ERD" width="50%" height="50%">


You will execute SQL queries to answer two questions, as listed in the instructions. Good luck!


In [6]:
-- highest_donation_assignments
WITH donation_details AS (
	SELECT
		d.assignment_id,
		dn.donor_type,
		ROUND(SUM(d.amount),2) AS rounded_total_donation_amount
	FROM
		donations AS d
	JOIN
		donors AS dn
	ON
		d.donor_id = dn.donor_id
	GROUP BY
		d.assignment_id,
		dn.donor_type
)
SELECT
	a.assignment_name,
	a.region,
	dd.rounded_total_donation_amount,
	dd.donor_type
FROM
	assignments AS a
JOIN
	donation_details AS dd
ON
	a.assignment_id = dd.assignment_id
ORDER BY
	dd.rounded_total_donation_amount DESC
LIMIT
	5;

,assignment_name,region,rounded_total_donation_amount,donor_type
0,Assignment_3033,East,3840.66,Individual
1,Assignment_300,West,3133.98,Organization
2,Assignment_4114,North,2778.57,Organization
3,Assignment_1765,West,2626.98,Organization
4,Assignment_268,East,2488.69,Individual


In [7]:
-- top_regional_impact_assignments
WITH donation_count AS (
	SELECT
		assignment_id,
		COUNT(donation_id) AS num_total_donations
	FROM
		donations
	GROUP BY
		assignment_id
),

assignment_rank AS (
	SELECT
		a.assignment_name,
		a.region,
		a.impact_score,
		dc.num_total_donations,
		ROW_NUMBER() OVER (PARTITION BY a.region ORDER BY a.impact_score DESC) AS rank_in_region
	FROM
		assignments AS a
	JOIN
		donation_count AS dc
	ON
		a.assignment_id = dc.assignment_id
	WHERE
		num_total_donations > 0
)

SELECT
	assignment_name,
	region,
	impact_score,
	num_total_donations
FROM
	assignment_rank
WHERE
	rank_in_region = 1
ORDER BY
	region;

,assignment_name,region,impact_score,num_total_donations
0,Assignment_316,East,10.00,2
1,Assignment_2253,North,9.99,1
2,Assignment_3547,South,10.00,1
3,Assignment_2794,West,9.99,2


In [8]:
SELECT * FROM assignments

,assignment_id,assignment_name,start_date,end_date,budget,region,impact_score
0,1,Assignment_1,2021-10-17,2021-12-04,-32322.03,West,5.55
1,2,Assignment_2,2020-10-26,2020-11-28,57278.40,South,1.45
2,3,Assignment_3,2021-08-11,2022-03-17,40414.51,West,2.34
3,4,Assignment_4,2021-11-22,2022-05-17,31732.48,East,7.05
4,5,Assignment_5,2020-11-22,2021-07-10,13548.22,North,5.29
...,...,...,...,...,...,...,...
4995,4996,Assignment_4996,2021-06-04,2022-02-15,-17276.64,West,5.85
4996,4997,Assignment_4997,2021-01-26,2021-08-03,-14681.98,South,4.68
4997,4998,Assignment_4998,2021-03-10,2021-12-05,-20391.76,West,3.46
4998,4999,Assignment_4999,2021-12-10,2022-07-22,9803.08,South,9.55


In [9]:
SELECT * FROM donations

,donation_id,donor_id,amount,donation_date,assignment_id
0,1,2733,271.36,2021-08-21 00:00:00+00:00,4226
1,2,2608,251.49,2021-10-15 00:00:00+00:00,1323
2,3,1654,528.38,2020-03-03 00:00:00+00:00,4881
3,4,3265,730.36,2021-02-06 00:00:00+00:00,1237
4,5,4932,285.96,2022-03-05 00:00:00+00:00,1626
...,...,...,...,...,...
4995,4996,4770,798.63,2020-03-08 00:00:00+00:00,4012
4996,4997,4093,693.00,2022-04-10 00:00:00+00:00,4700
4997,4998,2569,57.04,2021-11-04 00:00:00+00:00,2688
4998,4999,847,41.62,2022-11-29 00:00:00+00:00,1053


In [10]:
SELECT * FROM donors

,donor_id,donor_name,donor_type
0,1,Donor_1,Individual
1,2,Donor_2,Organization
2,3,Donor_3,Individual
3,4,Donor_4,Organization
4,5,Donor_5,Organization
...,...,...,...
4995,4996,Donor_4996,Individual
4996,4997,Donor_4997,Corporate
4997,4998,Donor_4998,Individual
4998,4999,Donor_4999,Individual
